In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

In [39]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 14 * 14, 128)
        self.fc2 = nn.Linear(128, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 14 * 14)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [40]:
model = CNN()

In [41]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [42]:
criterion = nn.CrossEntropyLoss()

In [43]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081))])

In [44]:
train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

In [45]:
from torch.utils.data import DataLoader

In [46]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=0.3081)
           )

In [47]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [48]:
test_dataloader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [49]:
def train(model, loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f"Train Epoche: {epoch} Loss: {loss.item():4f}")

In [ ]:
for (data, target) in train_dataloader:
    print(data)

In [ ]:
for (data, target) in train_dataloader:
    print(target)

In [52]:
for epoch in range(1, 6):
    train(model, train_dataloader, optimizer, criterion, epoch)

Train Epoche: 1 Loss: 2.306889
Train Epoche: 1 Loss: 0.101201
Train Epoche: 1 Loss: 0.079091
Train Epoche: 1 Loss: 0.098533
Train Epoche: 1 Loss: 0.076713
Train Epoche: 1 Loss: 0.087325
Train Epoche: 1 Loss: 0.026471
Train Epoche: 1 Loss: 0.118774
Train Epoche: 1 Loss: 0.007647
Train Epoche: 1 Loss: 0.052503
Train Epoche: 2 Loss: 0.002025
Train Epoche: 2 Loss: 0.122504
Train Epoche: 2 Loss: 0.016191
Train Epoche: 2 Loss: 0.082330
Train Epoche: 2 Loss: 0.012969
Train Epoche: 2 Loss: 0.002443
Train Epoche: 2 Loss: 0.005597
Train Epoche: 2 Loss: 0.002235
Train Epoche: 2 Loss: 0.131272
Train Epoche: 2 Loss: 0.004510
Train Epoche: 3 Loss: 0.098785
Train Epoche: 3 Loss: 0.007479
Train Epoche: 3 Loss: 0.002182
Train Epoche: 3 Loss: 0.000482
Train Epoche: 3 Loss: 0.081029
Train Epoche: 3 Loss: 0.008193
Train Epoche: 3 Loss: 0.001234
Train Epoche: 3 Loss: 0.091759
Train Epoche: 3 Loss: 0.035477
Train Epoche: 3 Loss: 0.003601
Train Epoche: 4 Loss: 0.001638
Train Epoche: 4 Loss: 0.000354
Train Ep

In [53]:
def test(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in loader:
            output = model(data)
            pred = output.argmax(dim=1)
            correct += (pred == target).sum().item()
            total += target.size(0)
    
    acc = 100 * correct / total
    print(f"Test accuracy: {acc:.2f}%")

In [56]:
test(model, test_dataloader)

Test accuracy: 98.56%
